<a href="https://colab.research.google.com/github/ELehmann91/Thesis_Multilingual_Transferlearning/blob/master/Text_Annotation_Tool_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

# ECOICOP Annotation
  
Use this tool to annotate your product texts with the correspondant eCoiCop category. 


## Get code from GitHub
  
At first we need to download the code from Github to here.

In [0]:
%%capture
!pip install eli5
!git clone 'https://github.com/ELehmann91/Thesis_Multilingual_Transferlearning'

%cd Thesis_Multilingual_Transferlearning
import labeler_cc5
import coicop_model
import pandas as pd
import numpy as np
from tqdm import tqdm
import io

## Upload from Google Drive

Loading CSV files stored in your google drive.

In [2]:
from google.colab import drive, files

drive.mount('/content/gdrive')
path ='/content/gdrive/My Drive/Thesis_ecb_ecoicop'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


The commands will bring you to a Google Authentication step. You should see a screen with Google Drive File Stream wants to access your Google Account. After you allow permission, copy the given verification code and paste it in the box in Colab.
In the notebook, click on the charcoal > on the top left of the notebook and click on Files. Locate the data folder you created earlier and find your data. Right-click on your data and select Copy Path. Store this copied path into a variable and you are ready to go.

Specify the exact path to your file and the seperator of your file to read it in.

In [3]:
import pandas as pd

data_path = '/data/'
file_path = 'at/norm_at.csv'
df = pd.read_csv(path+data_path+file_path,sep='|',index_col=False)
df[:2]

,Unnamed: 0,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,url_text,text,labeled_by,sort_columns
0,0,de,Megaspeed Scheibenreiniger Meeresbrise 5l,Haushalt Haushaltszubehör Autopflege,NaN,NaN,https://www.interspar.at/shop/lebensmittel/meg...,shop lebensmittel megaspeed scheibenreiniger m...,NaN,99_Non-Food,999_Non-Food,9999_Non-Food,11_Food,113_Fish and seafood,1133_Fresh or chilled seafood,Spar,NaN,6.99,2020002938956,shop lebensmittel megaspeed scheibenreiniger m...,Megaspeed Scheibenreiniger Meeresbrise 5lHaush...,Erik,65
1,1,de,Wolf Eigold Muscheln Muscheln 500G,Vorratsschrank Beilagen & Basisprodukte Te...,Eier aus Bodenhaltung,NaN,https://www.interspar.at/shop/lebensmittel/wol...,shop lebensmittel wolf eigold muscheln muscheln,NaN,11_Food,111_Bread and cereals,1116_Pasta products and couscous,11_Food,113_Fish and seafood,1133_Fresh or chilled seafood,Spar,NaN,1.89,121354,shop lebensmittel wolf eigold muscheln muscheln,Wolf Eigold Muscheln Muscheln 500GVorratsschra...,Erik,65


Print out the dataframe to see if the import is correct.

## Predict CoiCop Level 4
  
How does it work? - Word Embeddings (left)
* Words are translated in vectors which are learned represent the words
* Embeddings are a multidimensional space (often 300+ dimesnsion) and words which relate to each other are closer in this space
* Those spaces can be aligned for different languages
* The example shows a three dimensional space with embeddings for three different languages, the translation are idealy close neighbours and the distance between milk and cheese is closer than the distance betwenn milk and potato  
more information: https://towardsdatascience.com/introduction-to-word-embeddings-4cf857b12edc
  
How does it work? - Recurrent Neural Network (right)
* Now every word is represented by a vector and the input to the model is a sequence of vectors
* LSTM (Long Short Term Memory) Networks are good in solving sequential tasks, because they are able to remember information from previous states (words) and output a representation for the sentence 
* This representation is used to classify in to the coicop categories  
more information: https://towardsdatascience.com/a-beginners-guide-on-sentiment-analysis-with-rnn-9e100627c02e 




![embed](https://drive.google.com/uc?id=1AoleK5q47ZTkPCtvxDD6icEpcnLc9viE)

For the prediction you have to specify the names of certain columns in your dataframe which the tool will use.  
* name_col is the column with the product name (mandatory)
* category_col is the column with the category given by the supermarked (optional)
* url_col is the column with  the url to the product (optional)
* lang ist the language of the texts, supports 'de' and 'fr' (mandatory)
* label_cat5 is the column with labels,if labeld (optional)



In [11]:
# init predictor

new_prediction = False

name_col = 'name'
category_col = 'categ'
url_col = 'url'
lang = 'de'
label_col = 'cc5'

#coicop_model
CoiCop_Predictor = coicop_model.predictor(df
                                          , name_col
                                          , category_col
                                          , url_col
                                          , lang)
    
if new_prediction:
    CoiCop_Predictor.predict()
    df = CoiCop_Predictor.get_df()

using name, category and words in url as input
using german embeddings


  1%|          | 2/303 [00:00<00:24, 12.10it/s]

95% quantile no. of words per row is 21 (trained on 39)


100%|██████████| 303/303 [00:25<00:00, 11.72it/s]

predictions ready


## Explonation  
  
It shows the three best candidates for a product and highlights the words which contribute to the estimate (positiv: green, negative: red)
If you leave the the brackets empty it will show a random product. If you want the explonation for a particular product, you can enter the text.  
If you enter text it always needs to be in marks ''.

In [17]:
#text = None
text = 'wurst spezialitäten produkte big power pork chip bites suess pikant'

CoiCop_Predictor.tell_me_why(text)

Explanation(estimator="SGDClassifier(alpha=0.001, average=False, class_weight=None,\n              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,\n              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,\n              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',\n              power_t=0.5, random_state=RandomState(MT19937) at 0x7F2F3041AA98,\n              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,\n              warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='1127_Dried, salted or smoked meat', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[0] wurst', weight=4.624431055746104, std=None, value=1.0), FeatureWeight(feature='[9] pikant', weight=0.6426329393340224, std=None, value=1.0), FeatureWeight(feature='[5] pork', weight=0.1362897278107582, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-2.6083089022675865, std=None, value=1.0), FeatureWeight(feature='[3] big', weight=-0.19598352054956333, std=None, value=1.0), FeatureWeight(feature='[7] bites', weight=-0.1877068930694695, std=None, value=1.0), FeatureWeight(feature='[8] suess', weight=-0.17955932985816178, std=None, value=1.0), FeatureWeight(feature='[4] power', weight=-0.14923963791467798, std=None, value=1.0), FeatureWeight(feature='[6] chip', weight=-0.055869153088344954, std=None, value=1.0), FeatureWeight(feature='[1] spezialitäten', weight=-0.035741639884987124, std=None, value=1.0), FeatureWeight(feature='[2] produkte', weight=-0.017267243669015244, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.8147656527748014, score=1.9736774025890784, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='wurst spezialitäten produkte big power pork chip bites suess pikant', spans=[('wurst', [(0, 5)], 4.624431055746104), ('spezialitäten', [(6, 19)], -0.035741639884987124), ('produkte', [(20, 28)], -0.017267243669015244), ('big', [(29, 32)], -0.19598352054956333), ('power', [(33, 38)], -0.14923963791467798), ('pork', [(39, 43)], 0.1362897278107582), ('chip', [(44, 48)], -0.055869153088344954), ('bites', [(49, 54)], -0.1877068930694695), ('suess', [(55, 60)], -0.17955932985816178), ('pikant', [(61, 67)], 0.6426329393340224)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=4.581986304856664, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-2.6083089022675865, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='1128_Other meat preparations', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[9] pikant', weight=0.7156559288825135, std=None, value=1.0), FeatureWeight(feature='[4] power', weight=0.08062441719649689, std=None, value=1.0), FeatureWeight(feature='[5] pork', weight=0.031000734189658402, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-1.3726753481865157, std=None, value=1.0), FeatureWeight(feature='[0] wurst', weight=-0.5614519191636346, std=None, value=1.0), FeatureWeight(feature='[2] produkte', weight=-0.3830640834635059, std=None, value=1.0), FeatureWeight(feature='[1] spezialitäten', weight=-0.29614259974206975, std=None, value=1.0), FeatureWeight(feature='[6] chip', weight=-0.128019848159965, std=None, value=1.0), FeatureWeight(feature='[7] bites', weight=-0.07764556385483745, std=None, value=1.0), FeatureWeight(feature='[8] suess', weight=-0.07233985263802956, std=None, value=1.0), FeatureWeight(feature='[3] big', weight=-0.06924804802652912, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.0982737879386374, score=-2.133306182966418, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='wurst spezialitäten produkte big power pork chip bites suess pikant', spans=[('wurst', [(0, 5)], -0.5614519191636346), ('spezialit

# Annotation

For the annotation tool you have to specify the naes of certain columns in your dataframe which the tool will use.  
* Labeled_by takes your name and stores it after every item you labeld.  
* Text column 1 is the name of the column for the first line of text which will be displayed to help you labeling the product, usually this is the product name.  
* Text column 2 is the name of the column for the second line of text which will be displayed to help you labeling the product, this could be the category or the translation.  
* The URL column is the name of the column where the url of the scraped product is located (if it is in the dataframe). The url might be helpfull if you are unsure about the product nd want to look up the whole page on the website. If you do not have the url in your data you can leave it empty.  
* CoiCop 5 prediction is the column where the prediction is stored. The prediction will appear preselected in the dropdown menu.   
  
The data are sorted in a way that catogies with few labels come first, if predictions of those label are available.


In [0]:
# init labeler

labeled_by = 'Erik'
text_column_1 = 'name'
text_column_2 = 'categ'
url_column = 'url'
CoiCop_5_pred_col = 'cc5_pred'

CoiCop_Labeler = labeler_cc5.labeler( labeled_by
                        , df
                        , text_column_1
                        , text_column_2
                        , url_column
                        , CoiCop_5_pred_col)

Now you are ready to label, after executing the next line you will see:
* Text 1
* Text 2
* URL
* Dropdown with one category which is the prediction for the product. If you are dissatisfied with it, you can click on the dropdown and select the right one.
* Next-Button will skip this product
* Save-Button will annotate the selected category to your dataframe
  
If you finished labeling just jump to the next line.

In [13]:
CoiCop_Labeler.start_to_label()

The next line will print out your current progress of annotating the data.

In [14]:
CoiCop_Labeler.get_stats()

new labels: 0
in total 1054 of 30360 labeled ( 3.0 %)


Output the data including annotation.

In [15]:
df= CoiCop_Labeler.output_labels()
df[:1]

,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,url_text,text,labeled_by,sort_columns
0,de,Nativa Green Tea Lemon 0% Zucker,getraenke alkoholfreie getraenke eistee,None,NaN,https://www.billa.at/produkte/nativa-green-tea...,produkte nativa green tea lemon zucker,NaN,NaN,NaN,NaN,21_Alcoholic beverages,211_Spirits,2112_Alcoholic soft drinks,Billa,NaN,1.29,00-293994,produkte nativa green tea lemon zucker,Nativa Green Tea Lemon 0% Zuckergetraenke al...,NaN,70


## Save data to Google Drive

In [16]:
if True:
    print(path+data_path+file_path,'saved')
    df.to_csv(path+data_path+file_path,sep='|')

/content/gdrive/My Drive/Thesis_ecb_ecoicop/data/at/norm_at.csv saved
